## Funcion 

In [1]:
import pandas as pd 
import ta
import optuna
import time
from multiprocessing import Pool
import plotly.graph_objects as go
import requests
from io import StringIO
import numpy as np
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
import xgboost as xgb
from sklearn.metrics import confusion_matrix

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random=1234


In [3]:
url = "https://raw.githubusercontent.com/Rerris/Technical_Analysis_Proyect/main/data/aapl_1d_test.csv"


In [4]:
def get_data(url):
    response = requests.get(url, verify=True)
    data = pd.read_csv(StringIO(response.text))
    data=data["Close"]
    data=pd.DataFrame(data)
    data["Closet-1"]=data["Close"].shift(-1)
    data["Closet-2"]=data["Close"].shift(-2)
    rsi_indicator = ta.momentum.RSIIndicator(close=data['Close'], window=14)
    data['RSI'] = rsi_indicator.rsi()
    #MACD
    macd_indicator = ta.trend.MACD(close=data['Close'], window_slow=26, window_fast=12, window_sign=9)
    data['macd'] = macd_indicator.macd()
    data['macd_signal'] = macd_indicator.macd_signal()

    # Crear instancia del indicador EMA con períodos de 13 y 48
    ema_13_indicator = ta.trend.EMAIndicator(close=data['Close'], window=13)
    ema_48_indicator = ta.trend.EMAIndicator(close=data['Close'], window=48)
        
    # Calcular las EMAs 13 y 48 
    data['Ema 13'] = ema_13_indicator.ema_indicator()
    data['Ema 48'] = ema_48_indicator.ema_indicator()
    # Inicializar el indicador EMA con una ventana de 200 períodos
    ema_indicator = ta.trend.EMAIndicator(close=data['Close'], window=200)
    # Calcular la EMA 200

    data['Ema 200'] = ema_indicator.ema_indicator()
    data["Long"] = False
    data["Short"] = False

    for i in range(len(data["Close"])):

        if i + 10 < len(data["Close"]):
            if data["Close"][i] > data["Close"][i + 10]:
                data.loc[i, "Long"] = True
            elif data["Close"][i] < data["Close"][i + 10]:
                data.loc[i, "Short"] = True
    data=data
    return data
        
    

In [5]:
data=get_data(url).iloc[200:]

In [6]:
def logistic_regression_long(data):

    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data['Long']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inicializar el modelo de regresión logística
    model = LogisticRegression()

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Imprimir el informe de clasificación
    #print(classification_report(y_test, y_pred))

In [7]:
logistic_regression_long(data)

Accuracy: 0.5315315315315315


In [8]:
def logistic_regression_short(data):

    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data['Short']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inicializar el modelo de regresión logística
    model = LogisticRegression()

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Imprimir el informe de clasificación
    #print(classification_report(y_test, y_pred))

In [9]:
logistic_regression_short(data)

Accuracy: 0.5405405405405406


In [10]:
def support_vector_machine_long(data):

    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data['Long']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inicializar el clasificador SVM
    clf = svm.SVC(kernel='linear')  # Puedes cambiar el kernel según tus necesidades (lineal, polinomial, RBF, etc.)

    # Entrenar el modelo
    clf.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = clf.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)


In [11]:
support_vector_machine_long(data)

Accuracy: 0.5675675675675675


In [12]:
def support_vector_machine_short(data):

    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data['Short']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inicializar el clasificador SVM
    clf = svm.SVC(kernel='linear')  # Puedes cambiar el kernel según tus necesidades (lineal, polinomial, RBF, etc.)

    # Entrenar el modelo
    clf.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = clf.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

In [13]:
support_vector_machine_short(data)

Accuracy: 0.5405405405405406


In [14]:
def XGBoost_long(data):

    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data['Long']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convertir los datos a un formato específico para XGBoost (DMatrix)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Definir los parámetros del modelo
    params = {
        'objective': 'binary:logistic',  # Problema de clasificación binaria
        'eval_metric': 'logloss',  # Métrica de evaluación
        'eta': 0.1,  # Tasa de aprendizaje
        'max_depth': 6,  # Profundidad máxima del árbol
        'subsample': 0.8,  # Proporción de muestras utilizadas para entrenar cada árbol
        'colsample_bytree': 0.8  # Proporción de características utilizadas para entrenar cada árbol
    }

    # Entrenar el modelo
    num_round = 100  # Número de iteraciones de entrenamiento (número de árboles)
    model = xgb.train(params, dtrain, num_round)

    # Predecir en el conjunto de prueba
    y_pred_proba = model.predict(dtest)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred_proba]  # Convertir probabilidades en etiquetas binarias

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

In [15]:
XGBoost_long(data)

Accuracy: 0.8648648648648649


In [16]:
def XGBoost_short(data):

    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data['Short']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convertir los datos a un formato específico para XGBoost (DMatrix)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Definir los parámetros del modelo
    params = {
        'objective': 'binary:logistic',  # Problema de clasificación binaria
        'eval_metric': 'logloss',  # Métrica de evaluación
        'eta': 0.1,  # Tasa de aprendizaje
        'max_depth': 6,  # Profundidad máxima del árbol
        'subsample': 0.8,  # Proporción de muestras utilizadas para entrenar cada árbol
        'colsample_bytree': 0.8  # Proporción de características utilizadas para entrenar cada árbol
    }

    # Entrenar el modelo
    num_round = 100  # Número de iteraciones de entrenamiento (número de árboles)
    model = xgb.train(params, dtrain, num_round)

    # Predecir en el conjunto de prueba
    y_pred_proba = model.predict(dtest)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred_proba]  # Convertir probabilidades en etiquetas binarias

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

In [17]:
XGBoost_short(data)

Accuracy: 0.8738738738738738


In [18]:
data

,Close,Closet-1,Closet-2,RSI,macd,macd_signal,Ema 13,Ema 48,Ema 200,Long,Short
200,148.76,149.26,149.48,60.064144,-0.387048,-1.214695,144.366068,145.033400,138.376440,False,True
201,149.26,149.48,148.69,61.062292,0.029030,-0.965950,145.065201,145.205915,138.484734,False,True
202,149.48,148.69,148.64,61.518042,0.372237,-0.698313,145.695887,145.380367,138.594140,False,True
203,148.69,148.64,149.32,58.854117,0.573869,-0.443876,146.123617,145.515454,138.694596,False,True
204,148.64,149.32,148.85,58.680919,0.721314,-0.210838,146.483100,145.642987,138.793555,False,True
...,...,...,...,...,...,...,...,...,...,...,...
748,193.60,193.05,193.15,54.609704,2.636316,3.236689,194.600511,188.472310,177.514883,False,False
749,193.05,193.15,193.58,53.026881,2.306308,3.050613,194.379009,188.659155,177.669462,False,False
750,193.15,193.58,192.53,53.291960,2.029448,2.846380,194.203437,188.842454,177.823497,False,False
751,193.58,192.53,NaN,54.481471,1.823710,2.641846,194.114374,189.035824,177.980278,False,False


In [36]:
from sklearn.model_selection import cross_val_score

# Optimizing using "Long" as the target variable
X = data[['RSI', 'Ema 13', 'Ema 200']]
y = data['Long']

# Optimizing Logistic Regression
def optimize_logistic_regression(trial):
    C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
    max_iter = trial.suggest_int('max_iter', 100, 1000) # Maximum number of iterations
    
    model = LogisticRegression(C=C, max_iter=max_iter)
    scores = cross_val_score(model, X, y, cv=5) # 5-fold cross-validation
    
    return scores.mean()

# Optimizing SVM
def optimize_svm(trial):
    C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly']) # Kernel type
    
    model = svm.SVC(C=C, kernel=kernel)
    scores = cross_val_score(model, X, y, cv=5) # 5-fold cross-validation
    
    return scores.mean()

# Optimizing XGBoost
def optimize_xgboost(trial):
    eta = trial.suggest_loguniform('eta', 0.01, 0.1) # Learning rate
    max_depth = trial.suggest_int('max_depth', 3, 10) # Maximum depth of the tree
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0) # Subsample ratio of the training instances
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1.0) # Subsample ratio of columns when constructing each tree
    
    model = xgb.XGBClassifier(eta=eta, max_depth=max_depth, subsample=subsample, colsample_bytree=colsample_bytree)
    scores = cross_val_score(model, X, y, cv=5) # 5-fold cross-validation
    
    return scores.mean()

# Ojective function for Optuna
def objective(trial):
    model_name = trial.suggest_categorical('model', ['logistic_regression', 'svm', 'xgboost'])
    
    if model_name == 'logistic_regression':
        return optimize_logistic_regression(trial)
    elif model_name == 'svm':
        return optimize_svm(trial)
    elif model_name == 'xgboost':
        return optimize_xgboost(trial)

# Optimize hyperparameters using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100) # We decided to do 100 trials becasue we noticed that the the study convverges

# Get the best hyperparameters and corresponding score
best_params = study.best_trial.params
best_score = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)

# Print optimal hyperparameters
for param, value in best_params.items():
    print(f"Optimal {param}: {value}")


[I 2024-03-09 21:06:07,941] A new study created in memory with name: no-name-da67a71e-0421-4670-9efe-5d6847b835e3
/var/folders/zc/5nsc4kc52sn0_cv_67lvvnp00000gn/T/ipykernel_12701/1564197116.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
[I 2024-03-09 21:06:07,978] Trial 0 finished with value: 0.504029484029484 and parameters: {'model': 'svm', 'C': 0.041290032652895235, 'kernel': 'poly'}. Best is trial 0 with value: 0.504029484029484.


/var/folders/zc/5nsc4kc52sn0_cv_67lvvnp00000gn/T/ipykernel_12701/1564197116.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
[I 2024-03-09 21:06:08,010] Trial 1 finished with value: 0.4931203931203931 and parameters: {'model': 'svm', 'C': 0.637936402081715, 'kernel': 'rbf'}. Best is trial 0 with value: 0.504029484029484.
/var/folders/zc/5nsc4kc52sn0_cv_67lvvnp00000gn/T/ipykernel_12701/1564197116.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 0.01, 0.1) # Learning rate
/var/folders/zc/5nsc4kc52sn0_cv_67lvvnp00000gn/T/ipykernel_12701/1564197116.py:31: Fut

Best Hyperparameters: {'model': 'svm', 'C': 0.2244947753004637, 'kernel': 'rbf'}
Best Score: 0.5967567567567567
Optimal model: svm
Optimal C: 0.2244947753004637
Optimal kernel: rbf


In [38]:
# Optimizing using "Short" as the target variable
X = data[['RSI', 'Ema 13', 'Ema 200']]
y = data['Short']

# Optimizing Logistic Regression
def optimize_logistic_regression(trial):
    C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
    max_iter = trial.suggest_int('max_iter', 100, 1000) # Maximum number of iterations
    
    model = LogisticRegression(C=C, max_iter=max_iter)
    scores = cross_val_score(model, X, y, cv=5) # 5-fold cross-validation
    
    return scores.mean()

# Optimizing SVM
def optimize_svm(trial):
    C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly']) # Kernel type
    
    model = svm.SVC(C=C, kernel=kernel)
    scores = cross_val_score(model, X, y, cv=5) # 5-fold cross-validation
    
    return scores.mean()

# Optimizing XGBoost
def optimize_xgboost(trial):
    eta = trial.suggest_loguniform('eta', 0.01, 0.1) # Learning rate
    max_depth = trial.suggest_int('max_depth', 3, 10) # Maximum depth of the tree
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0) # Subsample ratio of the training instances
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1.0) # Subsample ratio of columns when constructing each tree
    
    model = xgb.XGBClassifier(eta=eta, max_depth=max_depth, subsample=subsample, colsample_bytree=colsample_bytree)
    scores = cross_val_score(model, X, y, cv=5) # 5-fold cross-validation
    
    return scores.mean()

# Ojective function for Optuna
def objective(trial):
    model_name = trial.suggest_categorical('model', ['logistic_regression', 'svm', 'xgboost'])
    
    if model_name == 'logistic_regression':
        return optimize_logistic_regression(trial)
    elif model_name == 'svm':
        return optimize_svm(trial)
    elif model_name == 'xgboost':
        return optimize_xgboost(trial)

# Optimize hyperparameters using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100) # We decided to do 100 trials becasue we noticed that the the study converges

# Get the best hyperparameters and corresponding score
best_params = study.best_trial.params
best_score = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)

# Print optimal hyperparameters
for param, value in best_params.items():
    print(f"Optimal {param}: {value}")


[I 2024-03-09 21:07:17,823] A new study created in memory with name: no-name-9eb1b34a-d5fc-4a49-ae33-aac876cabfc3
/var/folders/zc/5nsc4kc52sn0_cv_67lvvnp00000gn/T/ipykernel_12701/175821962.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 10) # Regularization parameter
[I 2024-03-09 21:07:17,856] Trial 0 finished with value: 0.5023095823095823 and parameters: {'model': 'logistic_regression', 'C': 0.243569660182636, 'max_iter': 311}. Best is trial 0 with value: 0.5023095823095823.
/var/folders/zc/5nsc4kc52sn0_cv_67lvvnp00000gn/T/ipykernel_12701/175821962.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguni

Best Hyperparameters: {'model': 'svm', 'C': 0.12794687806761568, 'kernel': 'rbf'}
Best Score: 0.5986732186732187
Optimal model: svm
Optimal C: 0.12794687806761568
Optimal kernel: rbf


In [20]:

def backtest_ml_strategy(data, model_func, target):
    # Prepare features and target variable
    X = data[['RSI', 'Ema 13', 'Ema 200']]
    y = data[target]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random)

    # Train the model
    model = model_func()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {target}: {accuracy}")

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for {target}:\n {cm}")

    # Calculate classification report
    cr = classification_report(y_test, y_pred)
    print(f"Classification Report for {target}:\n {cr}")

    return model

# Backtest for long positions
model_long = backtest_ml_strategy(data, LogisticRegression, 'Long')

# Backtest for short positions
model_short = backtest_ml_strategy(data, LogisticRegression, 'Short')


Accuracy for Long: 0.5585585585585585
Confusion Matrix for Long:
 [[53 11]
 [38  9]]
Classification Report for Long:
               precision    recall  f1-score   support

       False       0.58      0.83      0.68        64
        True       0.45      0.19      0.27        47

    accuracy                           0.56       111
   macro avg       0.52      0.51      0.48       111
weighted avg       0.53      0.56      0.51       111

Accuracy for Short: 0.6036036036036037
Confusion Matrix for Short:
 [[19 28]
 [16 48]]
Classification Report for Short:
               precision    recall  f1-score   support

       False       0.54      0.40      0.46        47
        True       0.63      0.75      0.69        64

    accuracy                           0.60       111
   macro avg       0.59      0.58      0.57       111
weighted avg       0.59      0.60      0.59       111

